# Functions

## Redifine numPryo functions

In [ ]:
#import numpyro as numpyro
#
## Get all names defined in the jnp
#all_names = dir(numpyro.distributions)
#class_dict = {name: getattr(numpyro.distributions, name) for name in all_names}
## Function to create new functions
#def create_new_function(class_obj):
#    def new_function(sample_shape=(), *args, **kwargs):
#        return numpyro.sample( class_obj(*args, **kwargs).expand(sample_shape))
#    
#    # Copy docstring from original function
#    if hasattr(class_obj, '__doc__'):
#        new_function.__doc__ = class_obj.__doc__
#
#    return new_function
#
## Create a Python file and write the functions to it
with open("NPgenerated_functions.py", "w") as file:
    file.write('import numpyro as numpyro'+'\n'+
                'from numpyro.distributions import*'+'\n'+
                'from numpyro.infer import MCMC, NUTS, Predictive'+'\n'+
                'from numpyro import sample'+'\n')
    for key, value in class_dict.items():
        if callable(value):
            try:
                # Create the new function using the class object from class_dict
                func_name = key.lower()
                function_str = f"def {func_name}(name, sample_shape=(), *args, **kwargs):\n"
                # Include the docstring from the original NumPyro function
                #function_str += f"    '''numpyro.distributions.{key.arg_constraints}'''\n"
                function_str += f"    return numpyro.sample(name, numpyro.distributions.{value.__name__}(*args, **kwargs).expand(sample_shape))\n"
                file.write(function_str)
            except Exception as e:
                print(f"Error creating function for {key}: {e}")
        else:
            print(f"Ignoring non-callable object for key {key}: {value}")

# Tests

In [9]:
import setup
setup.setup()
from Darray import*
from Mutils import*
from network import*
from numpyproLigth import*

jax.local_device_count 32


## Array operations

In [12]:
#x = jax.random.normal(jax.random.key(0), (8192, 8192))
repeated_structure = jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = jnp.tile(repeated_structure, (40, 4))
x

Array([[ 1,  2,  3, ...,  8,  9, 10],
       [ 1,  2,  3, ...,  8,  9, 10],
       [ 1,  2,  3, ...,  8,  9, 10],
       ...,
       [ 1,  2,  3, ...,  8,  9, 10],
       [ 1,  2,  3, ...,  8,  9, 10],
       [ 1,  2,  3, ...,  8,  9, 10]], dtype=int32)

In [3]:
# Example usage:
y = split_array_to_cores(x)
jax.debug.visualize_array_sharding(y)

  CPU 0    CPU 1    CPU 2    CPU 3  
                                    
  CPU 4    CPU 5    CPU 6    CPU 7  
                                    
  CPU 8    CPU 9   CPU 10   CPU 11  
                                    
 CPU 12   CPU 13   CPU 14   CPU 15  
                                    
 CPU 16   CPU 17   CPU 18   CPU 19  
                                    
 CPU 20   CPU 21   CPU 22   CPU 23  
                                    
 CPU 24   CPU 25   CPU 26   CPU 27  
                                    
 CPU 28   CPU 29   CPU 30   CPU 31  
                                    

In [4]:
jax.debug.visualize_array_sharding(jnp.sin(y))
jnp.sin(y) == jnp.sin(x)

  CPU 0    CPU 1    CPU 2    CPU 3  
                                    
  CPU 4    CPU 5    CPU 6    CPU 7  
                                    
  CPU 8    CPU 9   CPU 10   CPU 11  
                                    
 CPU 12   CPU 13   CPU 14   CPU 15  
                                    
 CPU 16   CPU 17   CPU 18   CPU 19  
                                    
 CPU 20   CPU 21   CPU 22   CPU 23  
                                    
 CPU 24   CPU 25   CPU 26   CPU 27  
                                    
 CPU 28   CPU 29   CPU 30   CPU 31  
                                    

Array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [5]:
jax.debug.visualize_array_sharding(y @ y)
y @ y == x@x 

  CPU 0    CPU 1    CPU 2    CPU 3  
                                    
  CPU 4    CPU 5    CPU 6    CPU 7  
                                    
  CPU 8    CPU 9   CPU 10   CPU 11  
                                    
 CPU 12   CPU 13   CPU 14   CPU 15  
                                    
 CPU 16   CPU 17   CPU 18   CPU 19  
                                    
 CPU 20   CPU 21   CPU 22   CPU 23  
                                    
 CPU 24   CPU 25   CPU 26   CPU 27  
                                    
 CPU 28   CPU 29   CPU 30   CPU 31  
                                    

Array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [6]:
jax.debug.visualize_array_sharding(jax.nn.softmax(y))
jax.nn.softmax(y) == jax.nn.softmax(x)

  CPU 0    CPU 1    CPU 2    CPU 3  
                                    
  CPU 4    CPU 5    CPU 6    CPU 7  
                                    
  CPU 8    CPU 9   CPU 10   CPU 11  
                                    
 CPU 12   CPU 13   CPU 14   CPU 15  
                                    
 CPU 16   CPU 17   CPU 18   CPU 19  
                                    
 CPU 20   CPU 21   CPU 22   CPU 23  
                                    
 CPU 24   CPU 25   CPU 26   CPU 27  
                                    
 CPU 28   CPU 29   CPU 30   CPU 31  
                                    

Array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [10]:
print(outstrength(x))
print(vec_outstrength(x))
print(para_outstrength(x))

[220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220]
[220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220]
[220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220 220
 220 220 220 220]


In [11]:
print(instrength(x))
print(vec_instrength(x))
print(para_instrength(x))

[ 40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240 280 320
 360 400  40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240
 280 320 360 400]
[ 40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240 280 320
 360 400  40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240
 280 320 360 400]
[ 40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240 280 320
 360 400  40  80 120 160 200 240 280 320 360 400  40  80 120 160 200 240
 280 320 360 400]


In [55]:
%%timeit
outstrength(x)

3.21 µs ± 45.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [56]:
%%timeit
para_outstrength(x)

1.08 ms ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [57]:
%%timeit
vec_instrength(x)

175 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [58]:
mask = x != 0
mask

Array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [59]:
print(indegree(x))
print(vec_indegree(x))
print(para_indegree(x))

[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]
[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]
[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]


In [60]:
print(outdegree(x))
print(vec_outdegree(x))
print(para_outdegree(x))

[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]
[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]
[40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40]


In [61]:
%%timeit
indegree(x)

6.77 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [62]:
%%timeit
vec_indegree(x)

177 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [63]:
%%timeit
para_indegree(x)

1.34 ms ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [64]:
%%memit 
outdegree(x)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


peak memory: 593.76 MiB, increment: 0.05 MiB


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [65]:
%memit
vec_indegree(x)

peak memory: 561.29 MiB, increment: 0.00 MiB


Array([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40], dtype=int32)

In [66]:
%memit
para_indegree(x)

peak memory: 561.29 MiB, increment: 0.00 MiB


Array([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40], dtype=int32)

## Network operations

## Rethinking

In [122]:
from NPgenerated_functions import*
import pandas as pd
import jax as jax
import jax.numpy as jnp
from numpyro import sample 

d = pd.read_csv('data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
weight = jnp.array(d.weight.values)
height = jnp.array(d.height.values)

def model(height, weight):
    s = uniform('s', [1], 0, 50)
    a = normal('a', [1], 178, 20)
    b = normal('b', [1], 0, 1)  
    sample("y", Normal(a + b * weight, s), obs=height)
dat = dict(height = height, weight = weight)

m = MCMC(NUTS(model), num_warmup=500, num_samples=500, num_chains=1)
m.run(jax.random.PRNGKey(0), **dat)
post = m.get_samples()

sample: 100%|██████████| 1000/1000 [00:00<00:00, 1304.87it/s, 7 steps of size 7.33e-01. acc. prob=0.92]


In [124]:
import arviz as az
data = az.from_numpyro(m)
az.summary(data=data)

arviz - WARNING - Shape validation failed: input_shape: (1, 500), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a[0],154.654,0.276,154.143,155.137,0.012,0.008,546.0,244.0,NaN
b[0],0.902,0.041,0.831,0.979,0.002,0.001,541.0,353.0,NaN
s[0],5.147,0.197,4.807,5.511,0.009,0.006,473.0,310.0,NaN


In [125]:
s = Uniform(0, 50).sample(jax.random.PRNGKey(0), (1,1))
a = Normal(178, 20).sample(jax.random.PRNGKey(0), (1,1))
b = Normal(0, 1).sample(jax.random.PRNGKey(0), (1,1))


In [ ]:
from jax.nn import softmax
def model(K, ni, y, i_ID, N):
    a =  numpyro.sample('a', dist.Normal(0,1).expand([K]))
    Sigma_individual = numpyro.sample("Sigma_individual", dist.Exponential(1).expand([ni]))
    L_individual =  numpyro.sample('L_individual', dist.LKJCholesky(ni, 50))
    z_individual = numpyro.sample("z_individual", dist.Normal(0, 1).expand([ni, K]))
    alpha = numpyro.deterministic("alpha", ((Sigma_individual[..., None] * L_individual) @ z_individual))

    # Apply softmax along the correct axis
    lk = numpyro.deterministic("alpha_softmax", softmax(a + alpha[i_ID], axis=1))
    numpyro.sample("y", dist.DirichletMultinomial(lk, 1), obs=y)

In [196]:
ni = 100
K = 3
a = Normal(0, 1).sample(jax.random.PRNGKey(0), (K,))
Sigma_individual = Exponential(1).sample(jax.random.PRNGKey(0), (ni,))
L_individual = LKJCholesky(ni, 1).sample(jax.random.PRNGKey(0), (1,))
z_individual = Normal(0, 1).sample(jax.random.PRNGKey(0), (ni,K))

In [197]:
number = ni
divisors = []

for i in range(1, number + 1):
    if number % i == 0:
        divisors.append(i)
divisors

[1, 2, 4, 5, 10, 20, 25, 50, 100]

In [206]:
(Sigma_individual[..., None] * L_individual).shape

(1, 100, 100)

In [207]:
z_individual.shape

(100, 3)

In [208]:
x = (Sigma_individual[..., None] * L_individual)[1,:,:]

In [249]:
cores = 25
s = PositionalSharding(mesh_utils.create_device_mesh(cores, devices = jax.devices()[0:cores]))
if cores is None:
    n = jax.local_device_count(backend=None)    
    shapes = jax.numpy.array(x.shape)
    if any( i < n  for i in shapes):
        cores = jax.numpy.min(shapes)
    else:
        cores = jax.local_device_count(backend=None)        
valid_shapes = valid_reshape_combinations(cores)
y = jax.device_put(x , s.reshape(valid_shapes[len(valid_shapes)-1]))

In [231]:
z_individual.shape

(100, 3)

In [246]:
s = PositionalSharding(mesh_utils.create_device_mesh(6, devices = jax.devices()[0:6])) #2. combine de cores nous avons besoin
y = jax.device_put(z_individual , s.reshape(2,3)) # 1. Comment la matrice peut etre divise 

In [247]:
%%timeit
((Sigma_individual[..., None] * L_individual) @ z_individual)

45.6 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [194]:
%%timeit
jax.numpy.matmul((Sigma_individual[..., None] * L_individual), z_individual)

42.7 µs ± 642 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [251]:
jax.debug.visualize_array_sharding(y)

                                             
  CPU 0    CPU 1    CPU 2    CPU 3    CPU 4  
                                             
                                             
  CPU 5    CPU 6    CPU 7    CPU 8    CPU 9  
                                             
                                             
 CPU 10   CPU 11   CPU 12   CPU 13   CPU 14  
                                             
                                             
 CPU 15   CPU 16   CPU 17   CPU 18   CPU 19  
                                             
                                             
 CPU 20   CPU 21   CPU 22   CPU 23   CPU 24  
                                             

In [252]:
jax.debug.visualize_array_sharding(z)

                                             
  CPU 0    CPU 1    CPU 2    CPU 3    CPU 4  
                                             
                                             
  CPU 5    CPU 6    CPU 7    CPU 8    CPU 9  
                                             
                                             
 CPU 10   CPU 11   CPU 12   CPU 13   CPU 14  
                                             
                                             
 CPU 15   CPU 16   CPU 17   CPU 18   CPU 19  
                                             
                                             
 CPU 20   CPU 21   CPU 22   CPU 23   CPU 24  
                                             

In [253]:
y @ z 

TypeError: dot_general requires contracting dimensions to have the same shape, got (100,) and (50,).

In [250]:
%%timeit
y @ z 

TypeError: dot_general requires contracting dimensions to have the same shape, got (100,) and (50,).